**Imports**

In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD,Adam
from keras.models import load_model
import random
nltk.download('punkt');
nltk.download('wordnet');

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\georg\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\georg\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


**Load data from Json file**

In [2]:
puncc = ['?', '!']
df = open('intents.json').read()
intents = json.loads(df)

In [3]:
words=[]
tags = []
documents = []
for i in intents['intents']:
    for p in i['patterns']:
        w = nltk.word_tokenize(p)
        words.extend(w)
        documents.append((w, i['tag']))
        if i['tag'] not in tags:
            tags.append(i['tag'])

**Pre Processing**

In [4]:
lemmatizer = WordNetLemmatizer()
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in puncc]
words = sorted(list(set(words)))
# sort classes
tags = sorted(list(set(tags)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(tags), "classes")
# words = all words, vocabulary
print (len(words), "unique lemmatized words")
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(tags,open('tags.pkl','wb'))

47 documents
9 classes
88 unique lemmatized words


In [5]:
train_df = []
output = [0] * len(tags)
for d in documents:
    # initialize  bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = d[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output)
    output_row[tags.index(d[1])] = 1
    
    train_df.append([bag, output_row])

In [6]:
random.shuffle(train_df);
train_df = np.array(train_df)
train_x = list(train_df[:,0])
train_y = list(train_df[:,1])

<ipython-input-6-829d164ae1d0>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_df = np.array(train_df)


### Model

In [7]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer= 'adam', metrics=['accuracy'])
hist = model.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

Epoch 1/100
10/10 [==============================] - 1s 2ms/step - loss: 2.1917 - accuracy: 0.1915
Epoch 2/100
10/10 [==============================] - 0s 2ms/step - loss: 2.2277 - accuracy: 0.1277
Epoch 3/100
10/10 [==============================] - 0s 2ms/step - loss: 2.1893 - accuracy: 0.1489
Epoch 4/100
10/10 [==============================] - 0s 2ms/step - loss: 2.2513 - accuracy: 0.1277
Epoch 5/100
10/10 [==============================] - 0s 2ms/step - loss: 2.1963 - accuracy: 0.0851
Epoch 6/100
10/10 [==============================] - 0s 2ms/step - loss: 2.1537 - accuracy: 0.1277
Epoch 7/100
10/10 [==============================] - 0s 2ms/step - loss: 2.1319 - accuracy: 0.1915
Epoch 8/100
10/10 [==============================] - 0s 2ms/step - loss: 2.0980 - accuracy: 0.1915
Epoch 9/100
10/10 [==============================] - 0s 2ms/step - loss: 2.0727 - accuracy: 0.2979
Epoch 10/100
10/10 [==============================] - 0s 2ms/step - loss: 2.0576 - accuracy: 0.1915
Epoch 11/

10/10 [==============================] - 0s 1ms/step - loss: 0.2520 - accuracy: 0.9362
Epoch 84/100
10/10 [==============================] - 0s 2ms/step - loss: 0.2675 - accuracy: 0.8936
Epoch 85/100
10/10 [==============================] - 0s 2ms/step - loss: 0.2613 - accuracy: 0.9362
Epoch 86/100
10/10 [==============================] - 0s 2ms/step - loss: 0.2422 - accuracy: 0.9574
Epoch 87/100
10/10 [==============================] - 0s 2ms/step - loss: 0.1712 - accuracy: 0.9787
Epoch 88/100
10/10 [==============================] - 0s 2ms/step - loss: 0.3299 - accuracy: 0.8936
Epoch 89/100
10/10 [==============================] - 0s 2ms/step - loss: 0.2549 - accuracy: 0.9362
Epoch 90/100
10/10 [==============================] - 0s 2ms/step - loss: 0.1681 - accuracy: 0.9787
Epoch 91/100
10/10 [==============================] - 0s 2ms/step - loss: 0.1340 - accuracy: 0.9787
Epoch 92/100
10/10 [==============================] - 0s 2ms/step - loss: 0.2707 - accuracy: 0.8723
Epoch 93/100


##### Load Model

In [8]:
model = load_model('chatbot_model.h5')
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('tags.pkl','rb'))

In [9]:
def clean_up_input(input):
    sentence_words = nltk.word_tokenize(input)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(input, words, show_details=True):

    sentence_words = clean_up_input(input)
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(input, model):
    # filter out predictions below a threshold
    p = bow(input, words,show_details=False)
    res = model.predict(np.array([p]),verbose=0)[0]
    
    results = [[i,r] for i,r in enumerate(res) ]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [10]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result


In [11]:
def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

## GUI

In [ ]:
#Creating GUI with tkinter
import tkinter
from tkinter import *


def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))

        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)

base = Tk()
base.title("Chatbot")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#19d2e9", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()